In [1]:
import time
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import svm
import ast
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

### Training on Chest X-ray14

In [3]:
#The file_list will contain the files that will be used for training on the Chest X-ray 14 dataset
#The test_list will contain the files that will be used for testing
file_list = []
test_list = []
#Path to the npz files that contain the feature representations
directory =  R'D:\SSD downloads\Processed Chest X-ray 14'

for filename in os.listdir(directory):
    #This stores the last 11 npz files for testing, otherwise they are added to the file_list
    for number in range(110, 121):
        if str(number) in filename:
            path = os.path.join(directory, filename)
            test_list.append(path)
        else:
            path = os.path.join(directory, filename)
            if os.path.isfile(path):
                file_list.append(path)

In [ ]:
#The classifiers list will contain the 14 different classifers for each different class
classifiers = []
#counter is used to stop the model training
counter = 0

#Loop over the npz files and load them, depending on what key they are stored as, they're loaded into the X_train or y_train 
#lists, once all the values from a npz file have been placed into the list, they are used to train all 14 classifers
for file in file_list:
    #These variables are reset for new file that is loaded
    loaded_array = np.load(file)
    train_image_data = []
    train_label = []
    #Checking what key the current value from the npz file has
    for key in loaded_array:
        if "Label" in key:
            train_label.append(loaded_array[key].tolist())
        else:
            train_image_data.append(loaded_array[key])    
    #Instantiate X_train and y_train
    y_train = train_label
    y_train = np.array(y_train)
    X_train = train_image_data
    #y_train.shape[1] = the number of classes
    #Loop over the classifiers if they exist train them using their index position within the list
    #If they don't exist train them and appende them to the list
    for i in range(y_train.shape[1]):
        if len(classifiers) > i:
            classifiers[i].fit(X_train, y_train[:,i])
        else:
            clf = logreg= LogisticRegression(solver = "lbfgs", C = 1.0, penalty = "l2", max_iter = 1000)
            clf.fit(X_train, y_train[:,i])
            classifiers.append(clf)
    #for every batch of 500 data that is used, counter increases by 1
    counter += 1
    print(counter)
    if counter == 20:
        break

### Testing on Chest X-ray14

In [87]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#counter is used to stop the model testing after x amount of iterations 
counter = 0
#AUC score will be stored in roc_auc variable
roc_auc = 0


for file in test_list:
    #These variables are reset for new file that is loaded
    X_test = []
    y_test = []
    loaded_array = np.load(file)
    #Checking what key the current value from the npz file has
    for key in loaded_array:
        if "Label" in key:
            y_test.append(loaded_array[key].tolist())
        else:
            X_test.append(loaded_array[key])    
    y_test = np.array(y_test)
    #The output of the probability predictions for the different classes will be stored in the y_prob variable
    y_prob = []
    #Generate all the probability predictions from all of the different classifiers
    for i in range(y_test.shape[1]):
        y_prob.append(classifiers[i].predict_proba(X_test)[:,1])
    y_prob = np.array(y_prob).T
    #Calculate the AUC-ROC for each label
    for i in range(y_test.shape[1]):
        #These values represent the indexs of [Effusion, Atelectasis, Consolidation, Cardiomegaly, Edema] in the y_prob[:,i] list
        if i in [2,3,8,7,12]:
            roc_auc += roc_auc_score(y_test[:, i], y_prob[:, i])
            print(f"AUC for label {i}: {roc_auc_score(y_test[:, i], y_prob[:, i])}")
    counter += 1
    if counter == 2:
        #AUC average score is calculated based of how many files have been iterated over
        roc_auc /= 5*counter
        break

AUC for label 2: 0.35915409290721156
AUC for label 3: 0.4914089347079038
AUC for label 7: 0.5419207317073171
AUC for label 8: 0.8965955284552846
AUC for label 12: 0.5331325301204819
AUC for label 2: 0.5379357484620643
AUC for label 3: 0.6159183673469388
AUC for label 7: 0.5935792349726776
AUC for label 8: 0.9791666666666666
AUC for label 12: 0.8631790744466801


In [88]:
#Average AUC score for in-distribution data
print(roc_auc)

0.6411990909793227

### Testing on CheXpert

In [89]:
#All the CheXpert files will be loaded into the test_list for out of distribution evaluation
test_list = []
directory =  R'Q:\Processed CheXpert'

for filename in os.listdir(directory):
    path = os.path.join(directory, filename)
    if os.path.isfile(path):
        test_list.append(path)
    #After 5 files have been added to test_list it breaks the loop
    if len(test_list) == 5:
        break

In [90]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#counter is used to stop the model testing after x amount of iterations 
counter = 0
#AUC score will be stored in roc_auc variable
roc_auc = 0


for file in test_list:
    #These variables are reset for new file that is loaded
    X_test = []
    y_test = []
    loaded_array = np.load(file)
    #Checking what key the current value from the npz file has
    for key in loaded_array:
        if "Label" in key:
            y_test.append(loaded_array[key].tolist())
        else:
            X_test.append(loaded_array[key])    
    y_test = np.array(y_test)
    #The output of the probability predictions for the different classes will be stored in the y_prob variable
    y_prob = []
    #Generate all the probability predictions from all of the different classifiers
    for i in range(y_test.shape[1]):
        y_prob.append(classifiers[i].predict_proba(X_test)[:,1])
    y_prob = np.array(y_prob).T
    #Calculate the AUC-ROC for each label
    for i in range(y_test.shape[1]):
        #These values represent the indexs of [Edema, Effusion, Atelectasis, Consolidation, Cardiomegaly] in the y_prob[:,i] list
        if i in [0,6,7,9,10]:
            roc_auc += roc_auc_score(y_test[:, i], y_prob[:, i])
            print(f"AUC for label {i}: {roc_auc_score(y_test[:, i], y_prob[:, i])}")
    counter += 1
    if counter == 2:
        #AUC score is calculated based of how many files have been iterated over
        roc_auc /= 5*counter
        break

AUC for label 0: 0.5107090871283011
AUC for label 6: 0.4786857974630901
AUC for label 7: 0.46519133241124944
AUC for label 9: 0.4702868852459016
AUC for label 10: 0.3944213923467036
AUC for label 0: 0.4769182782283219
AUC for label 6: 0.5368579746309003
AUC for label 7: 0.5373443983402489
AUC for label 9: 0.49955908289241624
AUC for label 10: 0.4191271498160851


In [91]:
#Average AUC score for out-of-distribution data
print(roc_auc)

0.4789101378503219